# Basic work

In [1]:
require('dotenv').config({ path: '../.env' });
var { evm, wallet, constants, common } = require('../utils');
var { toUnit } = require('../utils/bn');
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm');
var { getPastEvents } = require('../utils/contracts');
var { next, clear, unixToDate, bnToNumber } = require('../utils/jupyter');
var { ethers, web3, Web3, artifacts } = require('hardhat');
var moment = require('moment');
var Plot = require('plotly-notebook-js');
var { CreditRecorder } = require('../utils/credit-recorder');

clear();

In [ ]:
var creditRecorder = new CreditRecorder();
clear();

In [2]:
next(async () => {
    await evm.reset({
        jsonRpcUrl: process.env.MAINNET_HTTPS_URL,
        blockNumber: constants.FORK_BLOCK_NUMBER,
    });
});

In [3]:
var jobOwner, keeper;
next(async () => {
    [jobOwner, keeper] = await ethers.getSigners();
});

In [4]:
var keep3r, governance, keep3rV1, helper;
next(async () => {
    const data = await common.setupKeep3r();
    keep3r = data.keep3r;
    governance = data.governance;
    keep3rV1 = data.keep3rV1;
    helper = data.helper;
});

In [5]:
var job;
next(async () => {
    job = await common.createJobForTest(keep3r.address, jobOwner);
    keep3r.connect(governance).addJob(job.address);
});

In [6]:
var w3Keep3r;
next(async () => {
    const artifact = await artifacts.readArtifact('Keep3r');
    w3Keep3r = new web3.eth.Contract(artifact.abi, keep3r.address);
});

## Work, work, work

#### Liquidity setup

In [7]:
var kp3rWeth, kp3rWethWale;
next(async () => {
    kp3rWethWale = await wallet.impersonate(constants.RICH_KP3R_WETH_POOL_ADDRESS);
    kp3rWeth = await ethers.getContractAt('@openzeppelin/contracts/token/ERC20/IERC20.sol:IERC20', constants.KP3R_WETH_POOL_ADDRESS);
    await keep3r.connect(governance).approveLiquidity(kp3rWeth.address);
});

#### Keeper setup

In [8]:
next(async () => {
    await keep3r.connect(keeper).bond(constants.KP3R_V1_ADDRESS, 0);
    await evm.advanceTimeAndBlock(moment.duration(3, 'days').as('seconds'));
    await keep3r.connect(keeper).activate(constants.KP3R_V1_ADDRESS);
});

#### Add 100 LP tokens to job

In [9]:
next(async () => {
    const liquidityToAdd = toUnit(1);
    await kp3rWeth.connect(kp3rWethWale).transfer(jobOwner.address, liquidityToAdd);
    await kp3rWeth.connect(jobOwner).approve(keep3r.address, liquidityToAdd);
    await keep3r.connect(jobOwner).addLiquidityToJob(job.address, kp3rWeth.address, liquidityToAdd);
});

#### Work for 4 periods

In [10]:
next(async () => {
    const period = moment.duration(5, 'days').as('seconds');
    const timeToWork = period * 1;
    const startedToWorkAt = await getLatestBlockTimestamp();
    const maxRestHours = 72;
    
    await creditRecorder.record(job.address);
    await evm.advanceTimeAndBlock(moment.duration(maxRestHours, 'hours').as('seconds'));
    await creditRecorder.record(job.address);
    
    while ((await getLatestBlockTimestamp()) - startedToWorkAt < timeToWork) {
//         const workDifficulty = Math.ceil(Math.random() * 1);
//         const workRest = moment.duration(Math.ceil(Math.random() * 72), 'hours').as('seconds');
        
//         const workDifficulty = Math.ceil(Math.random() * 1);
        const workRest = moment.duration(maxRestHours, 'hours').as('seconds');
        
//         console.log(`Working with difficulty: ${workDifficulty}, sleeping ${moment.duration(workRest, 'seconds').as('hours')} hours`);
        console.log(`Sleeping ${moment.duration(workRest, 'seconds').as('hours')} hours`);
        await job.connect(keeper).work();
        await creditRecorder.record(job.address);
        
        await evm.advanceTimeAndBlock(workRest);
        await creditRecorder.record(job.address);
    }
});

#### Get data from events

In [11]:
var workTrace = { x: [], y: [] };

next(async () => {
    const workEvents = await getPastEvents(w3Keep3r, 'KeeperWork');
    const timestamps = workEvents.map(workEvent => getBlockTimestamp(workEvent.blockNumber));

    return Promise.all(timestamps).then(timestamp => {
        workTrace.x.push(unixToDate(timestamp));
        workTrace.y.push(0);
    });
});

## Draw plot

In [12]:
next(() => {
    var plot = Plot.createPlot([], { title: 'Basic work' });
    plot.addTraces([{
        ...creditRecorder.getPendingCredits(job.address),
        name: 'Pending credits',
        mode: 'lines',
        line: {
            color: 'rgba(0, 0, 0, .3)',
            width: 1,
            dash: 'dashdot'
        }
    }]);
    plot.addTraces([{
        ...creditRecorder.getCurrentCredits(job.address),
        name: 'Current credits',
        mode: 'lines',
        line: {
            color: 'rgba(51, 0, 255, .3)',
            width: 1,
            dash: 'dashdot'
        }
    }]);
    plot.addTraces([{
        ...creditRecorder.getTotalCredits(job.address),
        name: 'Total credits',
        mode: 'lines+markers',
        line: {
            color: 'rgb(63, 255, 0)',
            width: 2
        }
    }]);
    plot.addTraces([{
        ...workTrace,
        name: 'Work',
        mode: 'markers',
        marker: {
            symbol: 'x',
            size: 12,
            color: 'rgb(0, 0, 255)'
        }
    }]);
    
    $$html$$ = plot.render();
});

In [14]:
clear();

Sleeping 72 hours
